In [ ]:
#Aim is to pull records across 100,000s of games across random chessdotcom profiles who's ELO is in certain range

#count number of occurences of silly strings per Walshee's idea (e.g 69, Xxx, 420 etc)
#frequency of dumb names for different countries
#average elo per country


In [1]:
import os
import requests
import time
import datetime as dt
from datetime import datetime
from datetime import timedelta
import pandas as pd
import re
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import csv
import time 
import shlex
from random import randrange
from random import choices as rand_choices
from chessdotcom import get_player_games_by_month as month
from chessdotcom.client import get_country_players as country_players
import nest_asyncio
nest_asyncio.apply()
from IPython.display import display
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Togo (TG) vs Malta (MT)
#
country_list = ['TG', 'MT']
result_list = []
for x in country_list:
    player_list = country_players(x).json['players']
    len(player_list)
    player_list = rand_choices(player_list, k=round((len(player_list)/10)))
    result_list.append(player_list)

country_1, country_2 = result_list
result_list

201

1086

[['salimamedome',
  'rastalghul',
  'roksymick',
  'delnorphine',
  'tubyhoney',
  'serpentfirez',
  'iamlukaas',
  'temyr_ji',
  'nico00019',
  'sergeayinou',
  'bryan2819',
  'valames',
  'nanitimo',
  'colby_rey',
  'peterquill001',
  'ambrutal',
  'ianoasis',
  'girlwithlookingfrosome',
  'blackcoach',
  'tubyhoney'],
 ['zinarokcuoglu',
  'darrenvandit',
  'lewiszero426',
  'sebripard',
  'holyblunder',
  'rindionlaci',
  'eamonnmifsud',
  'luckygibby',
  'driptip',
  'idkrun',
  'jtuk12',
  'fbellic',
  'guidomassimo',
  'jangelo11',
  'toljab17',
  'blingting',
  'v-pebbles',
  'birwina',
  'johnvc55',
  'aklby',
  'sms_mal',
  'ungabunga2000',
  'ungabunga2000',
  'gdlikeschess',
  'misterstray',
  'egr471',
  'tim_grech',
  'fransgalour',
  'tamiturner15',
  'macstrogan',
  'davecodex',
  'linea26',
  'vukkurcina',
  'alesawy1985',
  'stefport',
  'eitangr1',
  'xalairras',
  'marchandise2022',
  'kev-mala',
  'lix6122',
  'karma158',
  'antochi90',
  'casha02',
  'thake188',
 

In [3]:
#Scraping functions

def count_all_games_in_year(NAME, first, last):
    active_players = NAME.copy()
    for name in NAME:
        for YEAR in range(first, last):
            for MONTH in range(1,13):
                if MONTH > datetime.now().month and YEAR == datetime.now().year:
                    break
                if MONTH == 1:
                    raw_details = str(month(name, YEAR, MONTH))
                    raw_file = open(f'.\\data_country_scrape\\{name}_{YEAR}.txt', "w", encoding="utf-8")
                    raw_file.write(str(raw_details))
                    raw_file.close()
                else:
                    raw_details = str(month(name, YEAR, MONTH))
                    raw_file = open(f'.\\data_country_scrape\\{name}_{YEAR}.txt', "a", encoding="utf-8")
                    raw_file.write(str(raw_details))
                    raw_file.close()

            raw_file = open(f'.\\data_country_scrape\\{name}_{YEAR}.txt', "r", encoding="utf-8")
            string_raw_file = raw_file.read()

            game_count = string_raw_file.count(f'username=\'{name}\'')

            if game_count == 0:
                active_players.remove(name)
                raw_file.close()

                os.remove(f'.\\data_country_scrape\\{name}_{YEAR}.txt')
                continue
            else:
                raw_file.close()
    return active_players

def json_parser(names, year):

    for name in names:
        filename = f'.\\data_country_scrape\\{name}_{year}.txt'
        with open(filename, 'r', encoding="utf-8") as chessfile:
            lines = chessfile.readlines()
            line = lines[0].replace('\\n', '\n')
            chessfile.close()
        with open(filename, 'w', encoding="utf-8") as outfile:
            outfile.write(line)
            outfile.close()

In [4]:
country_1_short = country_1[0:10]
country_2_short = country_2[0:10]

In [5]:
active_players_list = []
for element in result_list:
    active_players = count_all_games_in_year(element, 2018, 2019) #Only using active player's statistics
    active_players_list.append(active_players)
    json_parser(active_players, 2018) #making it more readable


    

In [6]:
active_players_1, active_players_2 = active_players_list

In [7]:
#Data processing/tabulation function
def process_data(names, year):
    Total_games = 0
    counter = 0
    for name in names:
        counter += 1
        with open(f'.\\data_country_scrape\\{name}_{year}.txt', 'r', encoding="utf-8") as infile:

            lines = infile.readlines()

            utc_date = None
            utc_time = None
            utc_endtime = ''
            UTC_DATE_KEY = '[UTCDate "'
            UTC_TIME_KEY = '[UTCTime "'
            termination = '[Termination'
            empty_data = {'time': [],
                          'end_time': [],
                          'result': [],
                          'colour': [],
                          'reason': [],
                          'white_elo': [],
                          'black_elo': [],
                          'opening': [],
                          'EC0': [],
                          'name': []
                          }

            final_data = {'time': [],
                          'end_time': [],
                          'result': [],
                          'colour': [],
                          'reason': [],
                          'white_elo': [],
                          'black_elo': [],
                          'opening': [],
                          'EC0': [],
                          'name': []
                          }

            temp_data = {'time': [],
                         'end_time': [],
                         'result': [],
                         'colour': [],
                         'reason': [],
                         'white_elo': [],
                         'black_elo': [],
                         'opening': [],
                         'EC0': [],
                         'name': []
                         }
            winsby = {'checkmate': 0, 'resignation': 0, 'time': 0, 'abandoned': 0}
            drawsby = {'repetition': 0, 'stalemate': 0, 'agreement': 0, 'insufficient material': 0,
                       'timeout vs insufficient material': 0, '50-move rule': 0}
            lossby = {'checkmate': 0, 'resignation': 0, 'time': 0, 'abandoned': 0}
            win_total = 0
            loss_total = 0
            draw_total = 0

            # Now we're going through all the games line by line (of the PGN format)

            for line in lines:
                fmt = '%Y.%m.%d %H:%M:%S'
                
                if f'White "{name}"' in line:
                    temp_data['colour'].append('White')
                if f'Black "{name}"' in line:
                    temp_data['colour'].append('Black')
                    tester = True 
                if line.startswith('[WhiteElo "'):
                    WHITE_ELO = line[len('[WhiteElo "'):-3]
                    temp_data['white_elo'].append(WHITE_ELO)
                if line.startswith('[BlackElo "'):
                    BLACK_ELO = line[len('[BlackElo "'):-3]
                    temp_data['black_elo'].append(BLACK_ELO)
                if line.startswith(UTC_DATE_KEY):
                    utc_date = line[len(UTC_DATE_KEY):-3]
                if line.startswith(UTC_TIME_KEY):
                    utc_time = line[len(UTC_TIME_KEY):-3]
                    timestamp = datetime.strptime(utc_date + ' ' + utc_time, fmt)
                    temp_data['time'].append(timestamp)
                if line.startswith('[EndTime "'):
                    utc_endtime = line[len('[EndTime "'):-3]
                    end_timestamp = datetime.strptime(utc_date + ' ' + utc_endtime, fmt)
                    temp_data['end_time'].append(end_timestamp)
                if line.startswith('[ECO "'):
                    ECO = line[len('[ECO "'):-3]
                    temp_data['EC0'].append(ECO)
                if line.startswith('[ECOUrl'):
                    opening = line[len('[ECOUrl "https://www.chess.com/openings/'):-3]
                    temp_data['opening'].append(opening)


                if ('time_control' in line) and ('time_control=\'600\'' not in line):
                    # reinitialise temp_data
                    temp_data = {'time': [],
                            'end_time': [],
                            'result': [],
                            'colour': [],
                            'reason': [],
                            'white_elo': [],
                            'black_elo': [],
                            'opening': [],
                            'EC0': [],
                            'name': []
                                }
                    continue 
                if ('time_control=\'600\'' in line):
                    # save data (temp) to final data, then reinitiliase data (temp)
                    if temp_data != empty_data:
                        if (temp_data['opening'] and temp_data['EC0']) == []:
                            temp_data['opening'].append('None')
                            temp_data['EC0'].append('None')
                        for i in temp_data:
                            final_data[i] = final_data[i] + temp_data[i]

                        temp_data = {'time': [],
                        'end_time': [],
                        'result': [],
                        'colour': [],
                        'reason': [],
                        'white_elo': [],
                        'black_elo': [],
                        'opening': [],
                        'EC0': [],
                        'name': []
                            }

            for line in lines:
                if termination in line:
                    if 'Game drawn by' in line:
                        reason_parts = shlex.split(line.strip('[]\n'))[1].split()
                        draw_reason = ' '.join(reason_parts[3:])
                        draw_total += 1
                        temp_data['reason'].append(draw_reason)
                        temp_data['result'].append('draw')
                    elif name in line:
                        if 'won' in line:
                            reason_parts = shlex.split(line.strip('[]\n'))[1].split()
                            reason = reason_parts[len(reason_parts) - 1]
                            temp_data['reason'].append(reason)
                            temp_data['result'].append('win')
                            win_total += 1
                    elif name not in line and 'won' in line:
                        reason_parts = shlex.split(line.strip('[]\n'))[1].split()
                        loss_reason = reason_parts[len(reason_parts) - 1]
                        temp_data['reason'].append(loss_reason)
                        temp_data['result'].append('loss')
                        loss_total += 1
                if ('time_control' in line) and ('time_control=\'600\'' not in line):
                # delete temp data
                    temp_data['name'].append(name)
                    temp_data = {'time': [],
                                'end_time': [],
                                'result': [],
                                'colour': [],
                                'reason': [],
                                'white_elo': [],
                                'black_elo': [],
                                'opening': [],
                                'EC0': [],
                                'name': []
                                }
                    continue 
                if 'time_control=\'600\'' in line:
                    # save temp_data to final data, then reinitiliase temp_data
                    temp_data['name'].append(name)
                    if temp_data != empty_data:
                        for i in temp_data:
                            final_data[i] = [*final_data[i], *temp_data[i]]

                        temp_data = {'time': [],
                        'end_time': [],
                        'result': [],
                        'colour': [],
                        'reason': [],
                        'white_elo': [],
                        'black_elo': [],
                        'opening': [],
                        'EC0': [],
                        'name': []
                                    }
        if counter == 1:
            chess_data = pd.DataFrame(final_data)
        else:
            chess_data_temp = pd.DataFrame(final_data)
            chess_data = pd.concat([chess_data, chess_data_temp])
            
    return chess_data


In [65]:
for n in chess_data_list:
    n.iloc[0][0]

Timestamp('2018-12-22 05:46:08')

Timestamp('2018-12-17 15:33:04')

In [8]:
active_players_1
active_players_2

['serpentfirez', 'sergeayinou', 'nanitimo', 'ianoasis']

['rindionlaci',
 'jtuk12',
 'ungabunga2000',
 'ungabunga2000',
 'stefport',
 'bulgarinstz',
 'vvv227',
 'narcis77',
 'theeewoooolf']

In [9]:
chess_data_1 = process_data(active_players_1, 2018)
chess_data_2 = process_data(active_players_2, 2018)
 


In [10]:
chess_data_1
chess_data_2
chess_data_list = [chess_data_1, chess_data_2]

,time,end_time,result,colour,reason,white_elo,black_elo,opening,EC0,name
0,2018-12-22 05:46:08,2018-12-22 05:47:58,loss,White,abandoned,738,939,Scandinavian-Defense-Mieses-Kotrc-Variation-3....,B01,serpentfirez
1,2018-12-24 06:42:06,2018-12-24 06:54:35,win,Black,checkmate,694,841,Queens-Pawn-Opening-Zukertort-Chigorin-Variation,D02,serpentfirez
2,2018-12-25 00:04:00,2018-12-25 00:10:13,loss,Black,resignation,845,753,Sicilian-Defense-Staunton-Cochrane-Variation,B20,serpentfirez
3,2018-12-26 06:12:17,2018-12-26 06:29:34,loss,White,checkmate,691,805,Scandinavian-Defense-Mieses-Kotrc-Variation-3....,B01,serpentfirez
4,2018-12-26 18:52:16,2018-12-26 18:55:02,win,White,checkmate,748,672,Kings-Pawn-Opening-Kings-Knight-Elephant-Gambi...,C40,serpentfirez
...,...,...,...,...,...,...,...,...,...,...
1,2018-05-20 11:55:24,2018-05-20 11:57:51,loss,White,abandoned,487,444,Englund-Gambit-Declined-Reversed-French-Variation,A40,nanitimo
2,2018-05-21 18:48:30,2018-05-21 19:01:01,loss,Black,checkmate,491,467,French-Defense-Marshall-Variation,C10,nanitimo
3,2018-05-21 19:02:53,2018-05-21 19:20:19,loss,Black,abandoned,441,446,French-Defense-Steinitz-Attack,C00,nanitimo
4,2018-09-15 01:40:41,2018-09-15 01:43:11,win,White,abandoned,471,384,Queens-Pawn-Opening-1...d5-2.e3,D00,nanitimo


,time,end_time,result,colour,reason,white_elo,black_elo,opening,EC0,name
0,2018-12-17 15:33:04,2018-12-17 15:36:48,loss,White,checkmate,1066,1332,Kings-Pawn-Opening-Leonardis-Variation,C20,rindionlaci
1,2018-12-17 15:37:28,2018-12-17 15:54:59,win,Black,resignation,854,1159,Queens-Pawn-Opening-Chigorin-Variation,D00,rindionlaci
2,2018-12-17 15:56:43,2018-12-17 16:10:02,loss,Black,abandoned,1308,1083,Bishops-Opening-Philidor-Counterattack,C23,rindionlaci
3,2018-12-20 12:51:07,2018-12-20 13:01:26,loss,White,checkmate,997,1106,Kings-Pawn-Opening-Leonardis-Variation,C20,rindionlaci
4,2018-12-20 13:02:45,2018-12-20 13:19:33,win,Black,time,1043,1083,Queens-Pawn-Opening-1...d5-2.e3,D00,rindionlaci
5,2018-12-20 13:19:47,2018-12-20 13:22:53,win,White,resignation,1136,1010,Queens-Pawn-Opening-1...d5-2.e3,D00,rindionlaci
6,2018-12-27 22:20:53,2018-12-27 22:24:01,loss,Black,abandoned,1246,1086,Bishops-Opening,C23,rindionlaci
0,2018-09-05 05:15:23,2018-09-05 05:23:02,loss,Black,checkmate,631,523,Philidor-Defense-3.Bc4,C41,jtuk12
1,2018-09-05 05:23:19,2018-09-05 05:34:28,loss,Black,checkmate,611,486,Kings-Pawn-Opening-Barnes-Defense,B00,jtuk12
2,2018-09-05 05:34:43,2018-09-05 05:36:26,win,Black,resignation,377,517,Reti-Opening-Arctic-Defense-2.e4,A04,jtuk12


In [11]:
for chess_data in chess_data_list:

    #converting elo values from str to int.
    chess_data[["white_elo", "black_elo"]] = chess_data[["white_elo", "black_elo"]].apply(pd.to_numeric)
    # print(type(chess_data.iloc[0,5]))
    #Checking no games are lower or higher elo than they should be
    mask = (chess_data['white_elo'] < 200) & (chess_data['white_elo'] > 4000)
    chess_data[mask]

,time,end_time,result,colour,reason,white_elo,black_elo,opening,EC0,name


,time,end_time,result,colour,reason,white_elo,black_elo,opening,EC0,name


In [12]:
for chess_data in chess_data_list:
    chess_data['game_length_s'] = chess_data.end_time - chess_data.time
    chess_data['game_length_s'] = chess_data['game_length_s'].apply(timedelta.total_seconds)
    chess_data['game_length_s'] = chess_data['game_length_s'].apply(int)

    #we made a game_length (seconds) column by taking difference of start and end times
    #We also converted game_length to minutes

In [47]:
#DATA CLEANING
#get rid of any weird minus game length times that shouldn't be there

mask_1 = (chess_data_1['game_length_s'] > 0) & (chess_data_1['game_length_s'] < 1201)
chess_data_1 = chess_data_1[mask_1]

mask_2 = (chess_data_2['game_length_s'] > 0) & (chess_data_2['game_length_s'] < 1201)
chess_data_2 = chess_data_2[mask_2]

In [80]:
#converting elo values from str to int.
chess_data_1[["white_elo", "black_elo"]] = chess_data_1[["white_elo", "black_elo"]].apply(pd.to_numeric)
chess_data_2[["white_elo", "black_elo"]] = chess_data_2[["white_elo", "black_elo"]].apply(pd.to_numeric)

C:\Users\tsadmin\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [40]:
chess_data_1 = chess_data_1[chess_data_1.game_length_m > 0]
chess_data_2 = chess_data_2[chess_data_2.game_length_m > 0]

In [48]:
#We've deleted some rows so let's reset the index count and delete the old index

chess_data_1.reset_index(inplace=True)
chess_data_1.drop(['index'], axis=1, inplace=True)
chess_data_1['game_id'] = (chess_data_1.index + 1) # let's put a game count column in that starts from 1

chess_data_2.reset_index(inplace=True)
chess_data_2.drop(['index'], axis=1, inplace=True)
chess_data_2['game_id'] = (chess_data_2.index + 1) # let's put a game count column in that starts from 1

C:\Users\tsadmin\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-48-dd0b92a9d087>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_data_1['game_id'] = (chess_data_1.index + 1) # let's put a game count column in that starts from 1


In [49]:
for chess_data in chess_data_list:
    chess_data.isnull().sum() #checking there are no null values
    chess_data.colour.value_counts() #count colour numbers to check nothing is off

time             0
end_time         0
result           0
colour           0
reason           0
white_elo        0
black_elo        0
opening          0
EC0              0
name             0
game_length_s    0
game_length_m    0
game_id          0
dtype: int64

White    217
Black    200
Name: colour, dtype: int64

time             0
end_time         0
result           0
colour           0
reason           0
white_elo        0
black_elo        0
opening          0
EC0              0
name             0
game_length_s    0
game_length_m    0
game_id          0
dtype: int64

Black    22
White    18
Name: colour, dtype: int64

In [50]:
chess_data_1

,time,end_time,result,colour,reason,white_elo,black_elo,opening,EC0,name,game_length_s,game_length_m,game_id
0,2018-12-22 05:46:08,2018-12-22 05:47:58,loss,White,abandoned,738,939,Scandinavian-Defense-Mieses-Kotrc-Variation-3....,B01,serpentfirez,110,1.83,1
1,2018-12-24 06:42:06,2018-12-24 06:54:35,win,Black,checkmate,694,841,Queens-Pawn-Opening-Zukertort-Chigorin-Variation,D02,serpentfirez,749,12.48,2
2,2018-12-25 00:04:00,2018-12-25 00:10:13,loss,Black,resignation,845,753,Sicilian-Defense-Staunton-Cochrane-Variation,B20,serpentfirez,373,6.22,3
3,2018-12-26 06:12:17,2018-12-26 06:29:34,loss,White,checkmate,691,805,Scandinavian-Defense-Mieses-Kotrc-Variation-3....,B01,serpentfirez,1037,17.28,4
4,2018-12-26 18:52:16,2018-12-26 18:55:02,win,White,checkmate,748,672,Kings-Pawn-Opening-Kings-Knight-Elephant-Gambi...,C40,serpentfirez,166,2.77,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,2018-05-20 11:55:24,2018-05-20 11:57:51,loss,White,abandoned,487,444,Englund-Gambit-Declined-Reversed-French-Variation,A40,nanitimo,147,2.45,399
399,2018-05-21 18:48:30,2018-05-21 19:01:01,loss,Black,checkmate,491,467,French-Defense-Marshall-Variation,C10,nanitimo,751,12.52,400
400,2018-05-21 19:02:53,2018-05-21 19:20:19,loss,Black,abandoned,441,446,French-Defense-Steinitz-Attack,C00,nanitimo,1046,17.43,401
401,2018-09-15 01:40:41,2018-09-15 01:43:11,win,White,abandoned,471,384,Queens-Pawn-Opening-1...d5-2.e3,D00,nanitimo,150,2.50,402


In [51]:
chess_data_1.describe()
chess_data_2.describe()

,white_elo,black_elo,game_length_s,game_length_m,game_id
count,403.000000,403.000000,403.000000,403.000000,403.000000
mean,547.826303,551.136476,667.131514,11.118933,202.000000
std,83.964365,82.340808,322.628763,5.377201,116.480327
min,310.000000,252.000000,3.000000,0.050000,1.000000
25%,497.500000,502.000000,405.000000,6.750000,101.500000
50%,551.000000,551.000000,691.000000,11.520000,202.000000
75%,597.000000,599.000000,954.500000,15.905000,302.500000
max,1222.000000,1147.000000,1200.000000,20.000000,403.000000


,white_elo,black_elo,game_length_s,game_length_m,game_id
count,40.000000,40.000000,40.000000,40.000000,40.000000
mean,780.550000,798.975000,685.675000,11.427500,20.500000
std,281.551648,284.907226,323.386756,5.390295,11.690452
min,377.000000,455.000000,74.000000,1.230000,1.000000
25%,537.750000,531.250000,487.250000,8.120000,10.750000
50%,734.000000,782.000000,761.000000,12.680000,20.500000
75%,988.000000,1060.500000,948.500000,15.805000,30.250000
max,1326.000000,1332.000000,1191.000000,19.850000,40.000000


In [52]:
chess_data_1.reason.value_counts(normalize=True)
chess_data_2.reason.value_counts(normalize=True)

checkmate                           0.446650
resignation                         0.213400
time                                0.181141
abandoned                           0.099256
stalemate                           0.029777
insufficient material               0.009926
timeout vs insufficient material    0.007444
repetition                          0.007444
agreement                           0.004963
Name: reason, dtype: float64

checkmate      0.425
resignation    0.275
time           0.200
abandoned      0.075
agreement      0.025
Name: reason, dtype: float64

In [57]:
chess_data_1.EC0.value_counts(normalize=True).head(5)
chess_data_2.EC0.value_counts(normalize=True).head(5)

A00    0.367246
B00    0.084367
B07    0.057072
A40    0.049628
D00    0.044665
Name: EC0, dtype: float64

D00    0.175
C20    0.100
A00    0.100
B00    0.100
C41    0.050
Name: EC0, dtype: float64

In [66]:
#Make column with first move
#ECO: B,C: e4
#ECO: A40 - A99, D, E: d4
#ECO: A00 : b4
#ECO: A01 : b3
#ECO: A02, A03 : f4
#ECO: A04-A09: Nf3
#ECO: A10-A39: c4

def EC0_first_move_mapping(x):
    if x[0:1] == 'B' or x[0:1] == 'C':
        return 'e4'
    if x[0:1] == 'D' or x[0:1] == 'E' or (x[0:1] == 'A' and int(x[1:]) >= 40 and int(x[1:]) <= 99):
        return 'd4'
    if x == 'A00':
        return 'b4'
    if x == 'A01':
        return 'b3'
    if x == 'A02' or x == 'A03':
        return 'f4'
    if x[0:1] == 'A' and int(x[1:]) >= 4 and int(x[1:]) <= 9:
        return 'Nf3'
    if x[0:1] == 'A' and int(x[1:]) >= 10 and int(x[1:]) <= 39:
        return 'c4'
    else:
        return 'unknown'

chess_data_1['first_move'] = chess_data_1['EC0'].apply(EC0_first_move_mapping)
chess_data_2['first_move'] = chess_data_2['EC0'].apply(EC0_first_move_mapping)

<ipython-input-66-223727908f0d>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_data_1['first_move'] = chess_data_1['EC0'].apply(EC0_first_move_mapping)


In [79]:
#Proportions of first moves for games for each country
chess_data_1.first_move.value_counts(normalize=True)
chess_data_2.first_move.value_counts(normalize=True)

b4     0.367246
e4     0.359801
d4     0.151365
f4     0.064516
Nf3    0.024814
b3     0.019851
c4     0.012407
Name: first_move, dtype: float64

e4     0.55
d4     0.25
b4     0.10
c4     0.05
Nf3    0.05
Name: first_move, dtype: float64

In [78]:
#Proportions of first moves AS WHITE for games for each country
opening_white_mask = chess_data_1['colour'] == 'White'
chess_data_1[opening_white_mask].first_move.value_counts(normalize=True)

b4     0.564593
e4     0.133971
f4     0.124402
d4     0.114833
Nf3    0.023923
b3     0.023923
c4     0.014354
Name: first_move, dtype: float64

In [ ]:
# average_game_length_1 = chess_data_1.game_length_m.mean()
# average_time_since_last_game_1 = chess_data_1.time_since_last_game_m.mean()
# average_games_per_day_1 = chess_data_1.game_id.count() / total_day_difference
# average_total_time_playing_per_day_1 =  average_games_per_day_1 * chess_data_1.game_length_m.mean()

In [82]:
def numerical_result(result):
    if result == 'loss':
        return -1
    elif result == 'win':
        return 1
    elif result == 'draw':
        return 0


In [87]:
chess_data_1['result'] = chess_data_1['result'].apply(numerical_result)
chess_data_2['result'] = chess_data_2['result'].apply(numerical_result)

<ipython-input-87-946e4697d13d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_data_1['result'] = chess_data_1['result'].apply(numerical_result)


In [94]:
user_elo_list = []
opponent_elo_list = []

for index, row in chess_data_1.iterrows():
    if row['colour'] == 'White':
        user_elo_list.append(row['white_elo'])
        opponent_elo_list.append(row['black_elo'])
    else:
        user_elo_list.append(row['black_elo'])
        opponent_elo_list.append(row['white_elo'])
        
user_elo = pd.Series(user_elo_list)
opponent_elo = pd.Series(opponent_elo_list)
chess_data_1['user_elo'] =  user_elo
chess_data_1['opponent_elo'] = opponent_elo
chess_data_1['elo_diff'] = chess_data_1['opponent_elo'] - chess_data_1['user_elo']

###############################################################
user_elo_list = []
opponent_elo_list = []

for index, row in chess_data_2.iterrows():
    if row['colour'] == 'White':
        user_elo_list.append(row['white_elo'])
        opponent_elo_list.append(row['black_elo'])
    else:
        user_elo_list.append(row['black_elo'])
        opponent_elo_list.append(row['white_elo'])
        
user_elo = pd.Series(user_elo_list)
opponent_elo = pd.Series(opponent_elo_list)
chess_data_2['user_elo'] =  user_elo
chess_data_2['opponent_elo'] = opponent_elo
chess_data_2['elo_diff'] = chess_data_2['opponent_elo'] - chess_data_2['user_elo']

In [102]:

chess_data_1.elo_diff.describe()
chess_data_2.elo_diff.describe()

<ipython-input-102-a3395e3572c8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chess_data_1['elo_diff'] = chess_data_1['opponent_elo'] - chess_data_1['user_elo']


count    403.000000
mean       5.761787
std       67.538991
min     -220.000000
25%      -35.000000
50%        3.000000
75%       45.500000
max      431.000000
Name: elo_diff, dtype: float64

count     40.000000
mean      13.975000
std      150.799831
min     -305.000000
25%      -94.250000
50%       21.000000
75%      110.500000
max      287.000000
Name: elo_diff, dtype: float64

In [101]:
chess_data_1

,time,end_time,result,colour,reason,white_elo,black_elo,opening,EC0,name,game_length_s,game_length_m,game_id,first_move,user_elo,opponent_elo
0,2018-12-22 05:46:08,2018-12-22 05:47:58,-1,White,abandoned,738,939,Scandinavian-Defense-Mieses-Kotrc-Variation-3....,B01,serpentfirez,110,1.83,1,e4,738,939
1,2018-12-24 06:42:06,2018-12-24 06:54:35,1,Black,checkmate,694,841,Queens-Pawn-Opening-Zukertort-Chigorin-Variation,D02,serpentfirez,749,12.48,2,d4,841,694
2,2018-12-25 00:04:00,2018-12-25 00:10:13,-1,Black,resignation,845,753,Sicilian-Defense-Staunton-Cochrane-Variation,B20,serpentfirez,373,6.22,3,e4,753,845
3,2018-12-26 06:12:17,2018-12-26 06:29:34,-1,White,checkmate,691,805,Scandinavian-Defense-Mieses-Kotrc-Variation-3....,B01,serpentfirez,1037,17.28,4,e4,691,805
4,2018-12-26 18:52:16,2018-12-26 18:55:02,1,White,checkmate,748,672,Kings-Pawn-Opening-Kings-Knight-Elephant-Gambi...,C40,serpentfirez,166,2.77,5,e4,748,672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,2018-05-20 11:55:24,2018-05-20 11:57:51,-1,White,abandoned,487,444,Englund-Gambit-Declined-Reversed-French-Variation,A40,nanitimo,147,2.45,399,d4,487,444
399,2018-05-21 18:48:30,2018-05-21 19:01:01,-1,Black,checkmate,491,467,French-Defense-Marshall-Variation,C10,nanitimo,751,12.52,400,e4,467,491
400,2018-05-21 19:02:53,2018-05-21 19:20:19,-1,Black,abandoned,441,446,French-Defense-Steinitz-Attack,C00,nanitimo,1046,17.43,401,e4,446,441
401,2018-09-15 01:40:41,2018-09-15 01:43:11,1,White,abandoned,471,384,Queens-Pawn-Opening-1...d5-2.e3,D00,nanitimo,150,2.50,402,d4,471,384


In [ ]:
############Analysis
#### Compare for both countries:
# 1a. Show top 5/bottom 5 opening moves and successes for each
# 1b. Show top 5/bottom 5 openings and successes for each

In [ ]:
# 2. Look at distribution of ELOs for each player

In [ ]:
# 3. Show proportions of reasons for each country - draw attention to likelihood of abandoning

In [ ]:
# 4. Compare game lengths for each country

In [ ]:
# 5. Compare average ELO for each country

In [ ]:
# 6. map out number of games vs ELO for each country to see which country is the quickest learner

In [ ]:
# 7. Look at the win rates for different days/hours/months
# 8. Look at game lengths for different days/hours/months

In [ ]:
# 9. Look at likelihood to win/draw/lose game depending on elo diff (refer to article - think regression problem)

In [ ]:
# 10. Use combined dataset to use features to predict game result (elo diff, colour)